In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas_ta
import pandas_datareader.data as web
import utils

In [ ]:
from utils.data_scraper import download_sp500_tickers 
sp500_list = download_sp500_tickers(save_local=True)

from utils.data_scraper import download_sp500_data

download_sp500_data(sp500_list, save_local=True)

In [ ]:
models = utils.models
data_scraper = utils.data_scraper

In [ ]:
df = data_scraper.get_sp500_data()
df.dropna(inplace=True)
df = df[df.index.get_level_values('ticker') != 'SPY']
sp500_list = data_scraper.get_sp500_tickers()
sp500_list = [ticker for ticker in sp500_list if ticker != 'SPY']

In [ ]:
df

In [ ]:
def safe_bbands(series, length=20):
	bb = pandas_ta.bbands(close=np.log1p(series), length=length)
	if bb is not None:
		return bb
	# Return a DataFrame of NaNs with the same index if bb is None
	return pd.DataFrame(np.nan, index=series.index, columns=['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0'])

df['garman_klass_volatility'] = ((np.log(df['high'] - np.log(df['low']) ** 2))/2 - (2 * np.log(2) - 1) * (np.log(df['adj_close']) - np.log(df['open']) ** 2))
df['rsi'] = df.groupby(level='ticker')['adj_close'].transform(lambda x: pandas_ta.rsi(x, length=20))
df['bb_low'] = df.groupby(level='ticker')['adj_close'].transform(lambda x: safe_bbands(x, length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level='ticker')['adj_close'].transform(lambda x: safe_bbands(x, length=20).iloc[:,1])
df['bb_high'] = df.groupby(level='ticker')['adj_close'].transform(lambda x: safe_bbands(x, length=20).iloc[:,2])

In [ ]:
# All 'close' prices for ticker 'AAPL' across all dates
df.xs('AAPL', level='ticker')['rsi'].plot()

In [ ]:
df.xs('A', level='ticker').head()

In [ ]:
def compute_atr(stock_data: pd.DataFrame, period: int = 14) -> pd.Series:
    atr = pandas_ta.atr(high=stock_data['high'], low=stock_data['low'], close=stock_data['adj_close'], length=period)
    return atr.sub(atr.mean()).div(atr.std())

In [ ]:
df = df.dropna()
df['atr'] = df.groupby(level='ticker', group_keys=False).apply(compute_atr)

In [ ]:
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

In [ ]:
df['macd'] = df.groupby(level='ticker', group_keys=False)['adj_close'].apply(compute_macd)

In [ ]:
df['dollar_volume'] = (df['volume'] * df['adj_close'])/1e6
df['close_over_open'] = np.log(df['adj_close'] / df['open'])

In [ ]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]
data = pd.concat([df.unstack(level='ticker')['dollar_volume'].resample('ME').mean().stack(level='ticker', future_stack=True).to_frame('dollar_volume'),
           df.unstack()[last_cols].resample('ME').last().stack(level='ticker', future_stack=True)], axis=1).dropna()

In [ ]:
data

In [ ]:
data['dollar_volume'] = (data.loc[:,'dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())
data['dollar_vol_rank'] = data.groupby(level='date')['dollar_volume'].rank(ascending=False)

In [ ]:
data = data[data['dollar_vol_rank'] <= 150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

**Calculate Monthly Returns for different time horizons as features**

In [ ]:
data

In [ ]:
def calculate_monthly_returns(df: pd.DataFrame, outlier_cutoffs: float = 0.005) -> pd.DataFrame:
    lags = [1, 2, 3, 6, 9, 12]
    
    for lag in lags:
        df[f'return_{lag}m'] = (df['adj_close'].pct_change(lag)
                                   .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoffs), upper=x.quantile(1 - outlier_cutoffs)))
                                   .add(1)
                                   .pow(1/lag)
                                   .sub(1)
                                   )
    return df

data = data.groupby(level='ticker', group_keys=False).apply(calculate_monthly_returns).dropna()


In [ ]:
data

Download Fama-French Factors and Calculate Rolling Betas

In [ ]:
utils.data_scraper.download_famafrench_data(save_local=True)

In [ ]:
factor_data = data_scraper.get_famafrench_data()
factor_data

In [ ]:
factor_data = factor_data.join(data['return_1m']).sort_index()
factor_data

In [ ]:
observations = factor_data.groupby(level='ticker').size()
valid_stocks = observations[observations >= 12]
factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]
factor_data

In [ ]:
from utils.models import calculate_betas

betas = calculate_betas(factor_data, method='rls')


In [ ]:
betas

In [ ]:
factors = ['mkt_rf', 'smb', 'hml', 'rmw', 'cma']
group_data = data.join(betas.groupby(level='ticker').shift())
group_data.loc[:, factors] = group_data.groupby(level='ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

In [ ]:
group_data = group_data.dropna()
group_data = group_data.drop('adj_close', axis=1)

In [ ]:
group_data.info()

In [ ]:
features = [
    'rsi', 'garman_klass_volatility',# 'close_over_open', #'atr', 'macd', 'bb_low', 'bb_mid', 'bb_high',
    'mkt_rf', 'smb', 'hml', 'rmw', 'cma'
]
cols = features + ['target_1m']
feature_data = group_data.copy()
feature_data['target_1m'] = feature_data.groupby('ticker')['return_1m'].shift(-1)
# convert infinities to NaN first (they also cause dropna)
feature_data = feature_data.replace([np.inf, -np.inf], np.nan)

null_counts = feature_data[cols].isnull().sum().sort_values(ascending=False)
null_counts_percent = (feature_data[cols].isnull().mean() * 100).round(2)
print("Null counts:")
print(null_counts)
print("\nPercent nulls:")
print(null_counts_percent)

In [ ]:
# features = [
#     'rsi', 'garman_klass_volatility',# 'close_over_open', #'atr', 'macd', 'bb_low', 'bb_mid', 'bb_high',
#     'mkt_rf', 'smb', 'hml', 'rmw', 'cma'
# ]
# cols = features + ['target_1m']
# feature_data = group_data.copy()
# feature_data['target_1m'] = feature_data.groupby('ticker')['return_1m'].shift(-1)
# model_df = feature_data.reset_index().dropna(subset=features + ['target_1m'])
# dates = sorted(model_df['date'].unique())
# for d in dates:
#     print(d)

In [ ]:
# Prepare supervised target and features (assumes `data` exists)
feature_data = group_data.copy()
feature_data['target_1m'] = feature_data.groupby('ticker')['return_1m'].shift(-1)

features = [
    'rsi', 'garman_klass_volatility', 'close_over_open',
    'atr', 'macd', 'bb_low', 'bb_mid', 'bb_high',
    'return_2m', 'return_3m', 'return_6m',
    'mkt_rf', 'smb', 'hml', 'rmw', 'cma'
]

# features = [
#     'rsi', 'garman_klass_volatility', 'atr', 'macd',
#     'bb_low', 'bb_mid', 'bb_high',
#     'return_1m', 'mkt_rf', 'smb', 'hml', 'rmw', 'cma'
# ]

model_df = feature_data.reset_index().dropna(subset=features + ['target_1m'])

from utils.models import rolling_train_predict_windowed
# Run the windowed rolling trainer
fixed_dates_pred, preds_df, last_model, scaler = rolling_train_predict_windowed(
    model_df, features,
    top_k=30, model_type='randomforest', window_months=12, min_train_rows=100
)

# Use this mapping for downstream optimisation / backtest
# Offset the dates by 1 day so they start at the beginning of each month
fixed_dates = {}
for d, tickers in fixed_dates_pred.items():
    # Parse date, add 1 day, then set to first day of the month
    new_date = (pd.to_datetime(d) + pd.DateOffset(days=1)).replace(day=1)
    fixed_dates[new_date.strftime('%Y-%m-%d')] = tickers
print(f'Created fixed_dates for {len(fixed_dates)} months')


# Optional: show last model interpretability
if last_model is not None:
    try:
        if hasattr(last_model, 'coef_'):
            coeffs = pd.Series(last_model.coef_, index=features).sort_values(ascending=False)
            print('Ridge coefficients (last model):')
            print(coeffs)
        elif hasattr(last_model, 'feature_importances_'):
            imps = pd.Series(last_model.feature_importances_, index=features).sort_values(ascending=False)
            print('Feature importances (last model):')
            print(imps)
    except Exception as e:
        print('Model inspection error:', e)
else:
    print('No model trained (insufficient history).')

In [ ]:
# Add next month predictions
future_investment_data = group_data.copy()
future_investment = future_investment_data.drop('return_1m', axis=1).groupby('ticker')

# print('feature_data info', feature_data.info())
future_model_df = future_investment_data.reset_index().dropna(subset=features)

last_date = future_model_df['date'].max()
print('last_date', last_date)
future_date = last_date + pd.DateOffset(days=1)
print(future_date)
pool = future_model_df[future_model_df['date'] == last_date].dropna(subset=features).copy()

X_pred = scaler.transform(pool[features])
y_pred = last_model.predict(X_pred)

pool = pool.assign(y_pred=y_pred)

selected = pool[pool['y_pred'] > 0].nlargest(10, 'y_pred')
fixed_dates[future_date.strftime('%Y-%m-%d')] = selected['ticker'].tolist()

In [ ]:
fixed_dates

In [ ]:
selected

In [ ]:
data

In [ ]:
fixed_dates

In [ ]:
stocks = feature_data.index.get_level_values('ticker').unique().tolist()

new_df = data_scraper.get_sp500_data(tickers=stocks,
                                     start_date=feature_data.index.get_level_values('date').unique()[0]-pd.DateOffset(months=12),
                                     end_date=feature_data.index.get_level_values('date').unique()[-1])
new_df = new_df.adj_close.unstack('ticker')

new_df

In [ ]:
fixed_dates

In [ ]:
from utils.portfolio import optimise_weights

returns_dataframe = np.log(new_df).diff().dropna()

portfolio_df = pd.DataFrame()

for start_date in fixed_dates.keys():
    
    try:
    
        end_date = (pd.to_datetime(start_date) + pd.offsets.MonthEnd(0)).strftime('%Y-%m-%d')
        
        cols = fixed_dates[start_date]
        
        optimisation_start_date = (pd.to_datetime(start_date) - pd.DateOffset(months=12)).strftime('%Y-%m-%d')
        
        optimisation_end_date = (pd.to_datetime(start_date) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        
        optimisation_df = new_df.loc[optimisation_start_date:optimisation_end_date]

        success = False
        try:
            
            weights = optimise_weights(prices=optimisation_df,
                                lower_bound=round(1/len(optimisation_df.columns)*0.5, 3))
            
            weights = pd.DataFrame(weights, index=pd.Series(0))
            
            success = True
        except Exception as e:
            weights = pd.DataFrame()
            print(f"Maximium Sharpe Ratio optimization failed for start date {start_date}: {e} continuing with equal weights for all tickers")
        
        
        if success == False:
            weights = pd.DataFrame([1/len(optimisation_df.columns) for i in range(len(optimisation_df.columns))],
                                   index=optimisation_df.columns.tolist(),
                                   columns=pd.Series(0)).T
        temp_df = returns_dataframe[start_date:end_date]
        
        temp_df = temp_df.stack(future_stack=True).to_frame('return').reset_index(level=0) \
            .merge(weights.stack(future_stack=True).to_frame('weight').reset_index(level=0, drop=True),
                left_index=True,
                    right_index=True) \
            .reset_index().set_index(['date', 'ticker']).unstack().stack(future_stack=True)
            
        temp_df.index.names = ['date' , 'ticker']
        
        temp_df['weighted_return'] = temp_df['return'] * temp_df['weight']
        
        temp_df = temp_df.groupby(level='date')['weighted_return'].sum().to_frame('strategy_return')
        
        portfolio_df = pd.concat([portfolio_df, temp_df])
    except Exception as e:
        print(f"An error occurred for start date {start_date}: {e}")
        

In [ ]:
portfolio_df.plot()

In [ ]:
spy_ret = data_scraper.get_sp500_data(tickers=['SPY'],
                                     start_date='2022-02-01',
                                     end_date='2025-11-01')
spy_ret = spy_ret.reset_index().drop('ticker', axis=1)
spy_ret['spy_ret'] = np.log(spy_ret['adj_close']).diff()
spy_ret = spy_ret[['date', 'spy_ret']]
spy_ret = spy_ret.set_index('date').dropna()
portfolio_df = portfolio_df.merge(spy_ret, left_index=True, right_index=True)

In [ ]:
portfolio_df.to_csv('data/portfolio_returns.csv')

In [ ]:
portfolio_df

In [ ]:
plt.style.use('ggplot')
portfolio_cumulative_return = np.exp(np.log1p(portfolio_df).cumsum())-1
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(14, 10))
portfolio_cumulative_return.plot(ax=ax)
plt.title("Cumulative Returns: RSI and Technical Indicators (BB, Volatility, MACD, ATR) vs SPY")
plt.ylabel("Cumulative Return")
plt.xlabel("Date")
plt.show()
